In [1]:
from translator import translate
from transformers import pipeline
import pandas as pd
import numpy as np
import os
from src.functions import count_labels,pii
import unittest
import sys
import operator
from tqdm import tqdm
import nltk


2022-04-27 10:51:31.432409: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-27 10:51:31.432441: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from datasets import load_dataset

dataset = load_dataset('xtreme', 'XNLI', split='test')
df=pd.DataFrame(dataset)
df

2022-04-27 10:51:35,520 - WARNING - download_and_prepare - Reusing dataset xtreme (/home/nour/.cache/huggingface/datasets/xtreme/XNLI/1.0.0/2fc6b63c5326cc0d1f73060649612889b3a7ed8a6605c91cecdbd228a7158b17) 


,language,sentence1,sentence2,gold_label
0,ar,حسنا ، لم أكن أفكر حتى حول ذلك ، لكن كنت محبطا...,لم أتحدث معه مرة أخرى.,contradiction
1,ar,حسنا ، لم أكن أفكر حتى حول ذلك ، لكن كنت محبطا...,كنت مستاء جدا لدرجة أنني بدأت بالحديث معه مرة ...,entailment
2,ar,حسنا ، لم أكن أفكر حتى حول ذلك ، لكن كنت محبطا...,دار بيننا حديث رائع.,neutral
3,ar,واعتقدت أن ذلك شرف لي ، ولا يزال ، ولايزال ، ك...,لم أكن على علم بأنني لم أكن الشخص الوحيد الذي ...,neutral
4,ar,واعتقدت أن ذلك شرف لي ، ولا يزال ، ولايزال ، ك...,كان لدي إنطباع أنني كنت الشخص الوحيد الذي لديه...,entailment
...,...,...,...,...
75145,zh,戴维森不应该用“scone”的发音来和“bone”押韵 - 无论如何不是因为维多利亚，而是他...,戴维森不应该以骨头和烤饼听起来一样的方式说话。,entailment
75146,zh,戴维森不应该用“scone”的发音来和“bone”押韵 - 无论如何不是因为维多利亚，而是他...,"如果戴维森押韵饼和骨头的话, 那就更好了。",contradiction
75147,zh,20万字的小说平均每8000字25美元。,价值25美元的20万字小说价格合理。,neutral
75148,zh,20万字的小说平均每8000字25美元。,"25美元的20万字的小说就是每美元4,000字。",contradiction


In [4]:
df_fr=df[df.language=='fr']
df_fr

,language,sentence1,sentence2,gold_label
30060,fr,"Eh bien, je ne pensais même pas à cela, mais j...",Je ne lui ai pas parlé de nouveau,contradiction
30061,fr,"Eh bien, je ne pensais même pas à cela, mais j...",J'étais si contrarié que je commençais juste à...,entailment
30062,fr,"Eh bien, je ne pensais même pas à cela, mais j...",Nous avons eu une grande discussion.,neutral
30063,fr,"Et je pensais que c'était un privilège, et ça ...",Je n'avais pas conscience que je n'étais pas l...,neutral
30064,fr,"Et je pensais que c'était un privilège, et ça ...",J'avais l'impression que j'étais le seul à avo...,entailment
...,...,...,...,...
35065,fr,Davidson ne devrait pas adopter la prononciati...,Davidson ne devrait pas parler d'une façon où ...,entailment
35066,fr,Davidson ne devrait pas adopter la prononciati...,Ce serait mieux que Davidson fasse rimer les m...,contradiction
35067,fr,Le roman moyen de 200 000 mots pour 25 $ fonct...,Un roman de 200 000 mots à 25 $ est un prix éq...,neutral
35068,fr,Le roman moyen de 200 000 mots pour 25 $ fonct...,Un roman de 200 000 payé 25 $ revient à 4 000 ...,contradiction


In [3]:
df_en=df[df.language=='en']
df_en

,language,sentence1,sentence2,gold_label
20040,en,"Well, I wasn't even thinking about that, but I...",I havent spoken to him again.,contradiction
20041,en,"Well, I wasn't even thinking about that, but I...",I was so upset that I just started talking to ...,entailment
20042,en,"Well, I wasn't even thinking about that, but I...",We had a great talk.,neutral
20043,en,"And I thought that was a privilege, and it's s...",I was not aware that I was not the only person...,neutral
20044,en,"And I thought that was a privilege, and it's s...",I was under the impression that I was the only...,entailment
...,...,...,...,...
25045,en,Davidson should not adopt the pronunciation of...,Davidson shouldn't talk in a way where bone an...,entailment
25046,en,Davidson should not adopt the pronunciation of...,It would be better if Davidson rhymed the word...,contradiction
25047,en,"The average novel of 200,000 words for $25 wor...","A 200,000 word novel at $25 is a fair price.",neutral
25048,en,"The average novel of 200,000 words for $25 wor...","A 200,000 word novel for $25 is 4,000 words pe...",contradiction


In [4]:
pipeline = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [5]:
# candidate_labels = ['First and last name', 'Mail','Internet protocol (IP) addresses', 'location', 'Date of birth', 'Date of death', 'Place of birth', 'Place of death', 'IP', 'a cookie', 'phone', 'health data','private data','personal data','not personal data','not private data']
name=["First and last name","full name", "person's name","first name and Family name","person"]
birth=["birthday"]
# ip=["Internet protocol (IP) addresses","IP adress"]
cookie=["cookie identifiers","a cookie"]
phone=["phone numbers","phone number","telphone number"]
mail=["mail addresses","mail address","mail"]
location=["location adress","private postal address"]
health=["health data","medical records","health information","medical information","Medical Procedure names","Medical Procedure identification codes"]
personal=["personal data","personal","personal information","private data","private information"]
not_personal=["not personal data","not personal information","not personal","not private data","not private","not private information"]
passport=['passport number','number']
Driving_license=['driving license','number']
social_security_number=['social security number']
Tax_file_number=['tax file number']
candidate_labels=name+cookie+phone+mail+location+health+personal+not_personal+birth+passport+Driving_license+social_security_number+Tax_file_number

In [7]:
text_for_test="""Marié à Michelle Robinson en 1992, Barack Obama entre en politique quatre ans plus tard : il est élu au Sénat de l'Illinois, où il effectue trois mandats, de 1997 à 2004. I
"""
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        text_translated=translate(txt)
        result_details=pipeline(text_translated, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        print(txt)
        print(result,pii(txt),'\n')

 50%|█████     | 1/2 [00:25<00:25, 25.22s/it]

Marié à Michelle Robinson en 1992, Barack Obama entre en politique quatre ans plus tard : il est élu au Sénat de l'Illinois, où il effectue trois mandats, de 1997 à 2004.
{'person': 0.8287651658058166, 'birth': 0.6673669219017029, 'personal': 0.6039626210927963, 'passport': 0.44271523505449295, 'Driving_license': 0.41577012836933136, 'location': 0.2945850007236004, 'not_personal': 0.24005366147806248, 'health': 0.12735427574565014, 'mail': 0.10251393169164658, 'phone': 0.10021297261118889, 'Tax_file_number': 0.08505287766456604, 'social_security_number': 0.07189692556858063, 'cookie': 0.05874930229038} {'Adresse': [('1992', ',', 'Barack', 'Obama')]} 



100%|██████████| 2/2 [00:47<00:00, 23.62s/it]

I
{'person': 0.6975784778594971, 'not_personal': 0.4704408844312032, 'personal': 0.4342993587255478, 'Driving_license': 0.4265482872724533, 'passport': 0.32948583364486694, 'location': 0.31704334914684296, 'birth': 0.26695480942726135, 'mail': 0.2541164830327034, 'Tax_file_number': 0.20225708186626434, 'health': 0.1854399765531222, 'cookie': 0.1510176695883274, 'phone': 0.12175271287560463, 'social_security_number': 0.11009800434112549} {} 



In [8]:
text_for_test="""un homme d'État américain. Il est le 44e président des États-Unis, en fonction du 20 janvier 2009 au 20 janvier 2017.
"""
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        text_translated=translate(txt)
        result_details=pipeline(text_translated, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        print(txt)
        print(result,pii(txt),'\n')

 50%|█████     | 1/2 [00:20<00:20, 20.77s/it]

un homme d'État américain.
{'person': 0.4656931638717651, 'not_personal': 0.21400641649961472, 'passport': 0.04818221554160118, 'birth': 0.03884506970643997, 'Driving_license': 0.03183781850384548, 'personal': 0.026590933557599783, 'location': 0.021148393396288157, 'Tax_file_number': 0.006720727309584618, 'mail': 0.004556481183196108, 'phone': 0.002631356823258102, 'health': 0.00107816838620541, 'social_security_number': 0.0010437247110530734, 'cookie': 0.00055667434935458} {} 



In [7]:
text_for_test=""" Toto TITI a fait de la natation pendant 5 ans, entre temps il a refait son passport de numero 215658125 au consulat
"""
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        text_translated=translate(txt)
        result_details=pipeline(text_translated, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        print(txt)
        print(result,pii(txt),'\n')

100%|██████████| 1/1 [00:19<00:00, 19.68s/it]

 Toto TITI a fait de la natation pendant 5 ans, entre temps il a refait son passport de numero 215658125 au consulat
{'person': 0.9293522715568543, 'passport': 0.838655024766922, 'Driving_license': 0.4903465658426285, 'location': 0.46587926760548726, 'birth': 0.317456990480423, 'personal': 0.27315772920846937, 'not_personal': 0.21899664402008057, 'health': 0.20923076394440918, 'phone': 0.16641171152393022, 'Tax_file_number': 0.05794449523091316, 'mail': 0.03665956794672335, 'social_security_number': 0.02658419869840145, 'cookie': 0.008137208642438054} {'phones': ['215658125'], 'Person': [' Toto TITI a']} 



In [8]:
text_for_test=""" la plaque d'immatriculation de ma voiture a été volé
"""
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        text_translated=translate(txt)
        result_details=pipeline(text_translated, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        print(txt)
        print(result,pii(txt),'\n')

100%|██████████| 1/1 [00:17<00:00, 17.56s/it]

 la plaque d'immatriculation de ma voiture a été volé
{'personal': 0.6000780433416366, 'Driving_license': 0.42422881722450256, 'not_personal': 0.29087536890680593, 'passport': 0.1350468187883962, 'person': 0.08061403278261423, 'location': 0.05328326336166356, 'birth': 0.0038977349177002907, 'cookie': 0.0034801810288627166, 'health': 0.0015346542277256958, 'Tax_file_number': 0.0005781728541478515, 'social_security_number': 0.0005394768668338656, 'phone': 0.0003918058840402712, 'mail': 0.0001226023353713875} {} 



In [9]:
text_for_test=""" le numero de la plaque d'immatriculation de ma voiture est AA-123-BB
"""
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        text_translated=translate(txt)
        result_details=pipeline(text_translated, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        print(txt)
        print(result,pii(txt),'\n')

100%|██████████| 1/1 [00:18<00:00, 18.12s/it]

 le numero de la plaque d'immatriculation de ma voiture est AA-123-BB
{'Driving_license': 0.88520747423172, 'passport': 0.49285547019098885, 'personal': 0.35022308081388476, 'birth': 0.21903681755065918, 'not_personal': 0.1511287366350492, 'person': 0.05518315786030144, 'location': 0.01616562981507741, 'health': 0.01562684114469448, 'cookie': 0.006189528896356933, 'mail': 0.0004047760715669331, 'social_security_number': 0.00021144619677215815, 'phone': 0.0001616334969488283, 'Tax_file_number': 0.00013160995149519295} {'license_plate': ['AA-123-BB']} 



In [10]:
text_for_test=""" you can find my passport in the back of my car, on the left side. You can call me on  +33(0)6 12 34 56 78
"""
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        text_translated=translate(txt)
        result_details=pipeline(text_translated, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        print(txt)
        print(result,pii(txt),'\n')

 50%|█████     | 1/2 [00:14<00:14, 14.47s/it]

 you can find my passport in the back of my car, on the left side.
{'personal': 0.5952373772859574, 'passport': 0.5229919850826263, 'Driving_license': 0.20772871864028275, 'person': 0.19082447774708272, 'location': 0.17996555007994175, 'not_personal': 0.149396694265306, 'birth': 0.029456239193677902, 'health': 0.017618491731506463, 'Tax_file_number': 0.002386688720434904, 'mail': 0.0009781087962134432, 'cookie': 0.0005673912601196207, 'phone': 0.0005044200903891275, 'social_security_number': 0.00025573268067091703} {} 



In [ ]:
text_for_test="""je travaille depuis 9 ans en tant qu'assistante commerciale puis chargée de clientèle au sein de l'entreprise de mobilier et décoration pour enfants Nanelle, collaborant avec des clients français mais aussi internationaux. \
 Aujourd'hui, je suis à la recherche d'un nouveau déni ou la dimension internationale est très presente, notamment avec les pays hispaniques. Forte de mes expériences passées, je sais être à l'écoute des clients, gérer les situations de crise et j'aime travailler en équipe.\
 Fort en 3 ans d'expérience en tant que responsable des projets événementiels et marketing opérationnel et 2 ans en tant qu'office manager au sein d'une start-up européenne, j'ai appris à mener des missions très diverses qui m'fait \
 permis de développer une importance capacité d'adaptation à toutes les situations. Aujourd'hui je suis à la recherche d'une nouvelle expérience d'office manager dans une strat-up afin de mettre à profit mes competences pour offrir un environnement \
 de travail simulant à mes futurs collegues.\
 Mon nom est Laval JACQUIN, j'ai 38 ans, j'habite 29 bis chemin de Rivals,\
 mon numéro de téléphone est le +33652105579 et la plaque d'immatriculation de ma voiture est BP-152-FX, mon adresse mail est le l.jaquin@okp4.com\
Issue a Civil Monetary Penalty notice \
Leading to a fine ranging up to £500,000\
Issue an Enforcement Notice \
A formal notice requiring an organisation or individual to take the action specified in the notice in order to bring about compliance \
with the Act and related laws. Failure to comply with a notice is a criminal offence.\
Request and agree an Undertaking with the organisation \
A formal undertaking can be given by an organisation to the ICO, committing the organisation to a particular course of action or \
otherwise achieving compliance.\
Mohandas Karamchand Gandhi est né le 2 octobre 1869 à Porbandar, dans l'actuel État du Gujarat, en Inde. Gandhi est né et a vécu \
toute sa vie en tant qu'hindou, mais dans une famille ouverte aux autres communautés religieuses, qu'elles soient jaïne, musulmane, ou parsie."""


pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        text_translated=translate(txt)
        result_details=pipeline(text_translated, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        print(txt)
        print(result,pii(txt),'\n')

 12%|█▎        | 1/8 [00:26<03:02, 26.05s/it]

je travaille depuis 9 ans en tant qu'assistante commerciale puis chargée de clientèle au sein de l'entreprise de mobilier et décoration pour enfants Nanelle, collaborant avec des clients français mais aussi internationaux.
{'not_personal': 0.30824191495776176, 'personal': 0.2822329627039532, 'person': 0.2510039895772934, 'phone': 0.11776222297921776, 'location': 0.10810620772341888, 'birth': 0.02412388753145933, 'passport': 0.0174852441996336, 'mail': 0.008737770956940949, 'cookie': 0.003342007054015994, 'Tax_file_number': 0.0027333921752870083, 'Driving_license': 0.0013418748276308179, 'social_security_number': 0.00019565093680284917, 'health': 0.00016902118780611394} {} 



 25%|██▌       | 2/8 [00:52<02:39, 26.54s/it]

Aujourd'hui, je suis à la recherche d'un nouveau déni ou la dimension internationale est très presente, notamment avec les pays hispaniques.
{'not_personal': 0.5814041271805763, 'phone': 0.15435394290834664, 'person': 0.10408069379627705, 'health': 0.03617899585515261, 'Tax_file_number': 0.028765035793185234, 'birth': 0.020488301835333306, 'mail': 0.017891529947519302, 'passport': 0.016267110593616962, 'social_security_number': 0.01242753304541111, 'personal': 0.008565973742709806, 'Driving_license': 0.007624065037816763, 'location': 0.0022149555443320423, 'cookie': 0.0016663727292325348} {} 



 38%|███▊      | 3/8 [01:18<02:10, 26.13s/it]

...

Mon nom est Laval JACQUIN, j'ai 38 ans, j'habite 29 bis chemin de Rivals, mon numéro de téléphone est le +33652105579 et la plaque d'immatriculation de ma voiture est BP-152-FX, mon adresse mail est le l.jaquin@okp4.comIssue a Civil Monetary Penalty notice Leading to a fine ranging up to £500,000Issue an Enforcement Notice A formal notice requiring an organisation or individual to take the action specified in the notice in order to bring about compliance with the Act and related laws.
{'person': 0.9767806768417359, 'Driving_license': 0.9112911224365234, 'phone': 0.8105910420417786, 'personal': 0.6570644111682972, 'location': 0.5809330008924007, 'birth': 0.5057771404584249, 'mail': 0.38115153710047406, 'not_personal': 0.32255076461782056, 'passport': 0.3193279579281807, 'cookie': 0.22782474849373102, 'Tax_file_number': 0.19343586266040802, 'health': 0.03966827830299735, 'social_security_number': 0.0027964420150965452} {'license_plate': ['BP-152-FX'], 'phones': ['+33652105579'], 'emails'

 88%|████████▊ | 7/8 [03:20<00:31, 31.18s/it]

Failure to comply with a notice is a criminal offence.Request and agree an Undertaking with the organisation A formal undertaking can be given by an organisation to the ICO, committing the organisation to a particular course of action or otherwise achieving compliance.Mohandas Karamchand Gandhi est né le 2 octobre 1869 à Porbandar, dans l'actuel État du Gujarat, en Inde.
{'birth': 0.8778972029685974, 'person': 0.7851595520973206, 'location': 0.1859237796937426, 'not_personal': 0.143127481918782, 'phone': 0.10411309581249953, 'passport': 0.06145062483847141, 'Tax_file_number': 0.04389606788754463, 'mail': 0.03502973665793737, 'personal': 0.03146217639247576, 'health': 0.020481907296925783, 'Driving_license': 0.01061034481972456, 'social_security_number': 0.009559129364788532, 'cookie': 0.00723938609007746} {'Person': [' Karamchand Gandhi est'], 'Adresse': [('1869', 'à', 'Porbandar', ',')]} 



100%|██████████| 8/8 [03:44<00:00, 28.02s/it]

Gandhi est né et a vécu toute sa vie en tant qu'hindou, mais dans une famille ouverte aux autres communautés religieuses, qu'elles soient jaïne, musulmane, ou parsie.
{'birth': 0.8661070068677267, 'person': 0.6726939827203751, 'location': 0.4058381952345371, 'personal': 0.397697323312362, 'phone': 0.20421574893407524, 'not_personal': 0.1376817519776523, 'passport': 0.07598704006522894, 'mail': 0.01777647389099002, 'Driving_license': 0.0061357831582427025, 'Tax_file_number': 0.004960409831255674, 'health': 0.004444622551091015, 'cookie': 0.0022649350139545277, 'social_security_number': 0.0008060909458436072} {} 



In [7]:

pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for row in  tqdm((df_fr['sentence2'][10:20]),total=10):
    txt=nltk.sent_tokenize(row)
    for sent in txt:
        text_translated=translate(sent)
        result_details=pipeline(text_translated, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        print(sent)
        print(result,pii(sent),'\n')

 10%|█         | 1/10 [00:20<03:00, 20.03s/it]

Je n'en parlerai pas, même s'il y a beaucoup à couvrir.
{'not_personal': 0.9841677049795786, 'personal': 0.9473645091056824, 'health': 0.47398141026496887, 'location': 0.38356031477451324, 'person': 0.32798192352056504, 'Driving_license': 0.26537295430898666, 'passport': 0.24798356369137764, 'mail': 0.22184817989667258, 'Tax_file_number': 0.18324404954910278, 'phone': 0.1756570041179657, 'birth': 0.16620522737503052, 'social_security_number': 0.13045403361320496, 'cookie': 0.12900198064744473} {} 



 20%|██        | 2/10 [00:39<02:39, 19.91s/it]

Je ne vais pas parler de l'histoire de la ville parce qu'il y aurait trop de choses à dire.
{'not_personal': 0.551948552330335, 'personal': 0.2628419755026698, 'location': 0.045987945050001144, 'person': 0.03468801584094763, 'passport': 0.028501528315246105, 'Driving_license': 0.027863468509167433, 'phone': 0.005101692668783168, 'mail': 0.004793880507349968, 'birth': 0.0038952240720391273, 'Tax_file_number': 0.0033899133559316397, 'social_security_number': 0.0018296389607712626, 'cookie': 0.0012604718212969601, 'health': 0.00108249699890924} {} 



 30%|███       | 3/10 [00:59<02:18, 19.83s/it]

Nous nous foutions de mettre de l'argent de côté
{'not_personal': 0.5260385796427727, 'Driving_license': 0.28527082223445177, 'passport': 0.2837647902779281, 'personal': 0.27040012553334236, 'person': 0.188962160050869, 'Tax_file_number': 0.1245325431227684, 'location': 0.0981641411781311, 'birth': 0.03875095024704933, 'phone': 0.02910542270789544, 'mail': 0.02597430596748988, 'social_security_number': 0.021740710362792015, 'cookie': 0.017067363136447966, 'health': 0.01395665000503262} {} 



In [7]:

pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for row in  tqdm((df['sentence2'][10:20]),total=10):
    txt=nltk.sent_tokenize(row)
    for sent in txt:
        text_translated=translate(sent)
        result_details=pipeline(text_translated, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        print(sent)
        print(result,pii(sent),'\n')

 10%|█         | 1/10 [00:21<03:15, 21.67s/it]

لن أتحدث عن ذلك ، على الرغم من أن هناك الكثير لتغطيته.
{'not_personal': 0.9883920550346375, 'personal': 0.9265740036964416, 'health': 0.370786115527153, 'location': 0.3156248480081558, 'person': 0.2849990278482437, 'Driving_license': 0.1955683995038271, 'passport': 0.18573249503970146, 'Tax_file_number': 0.15892036259174347, 'mail': 0.12784380465745926, 'birth': 0.10817523300647736, 'phone': 0.08716536561648051, 'social_security_number': 0.08305728435516357, 'cookie': 0.08057179534807801} {} 



 20%|██        | 2/10 [00:42<02:47, 20.99s/it]

لن أتحدث عن تاريخ المدينة لأن هناك الكثير مما يمكن قوله حولها.
{'not_personal': 0.8378292421499888, 'personal': 0.25604716744273903, 'location': 0.053988548927009106, 'Driving_license': 0.03388188756071031, 'passport': 0.033841521362774074, 'person': 0.027179042622447015, 'phone': 0.00490471573236088, 'mail': 0.004216025350615382, 'birth': 0.002978001721203327, 'Tax_file_number': 0.002551990794017911, 'health': 0.0014568092204475154, 'cookie': 0.0013126576668582857, 'social_security_number': 0.0008596362313255668} {} 



 30%|███       | 3/10 [01:02<02:25, 20.73s/it]

...

لا أعرف لماذا حدث هذا.
{'personal': 0.38076911866664886, 'not_personal': 0.3781730408469836, 'Driving_license': 0.24959032237529755, 'person': 0.2415455587208271, 'passport': 0.20134895108640194, 'location': 0.14183686673641205, 'mail': 0.12671119223038355, 'Tax_file_number': 0.10870002210140228, 'health': 0.08746906804541747, 'cookie': 0.07559278607368469, 'phone': 0.06568943212429683, 'social_security_number': 0.03929833695292473, 'birth': 0.03066185489296913} {} 



 90%|█████████ | 9/10 [03:09<00:20, 20.80s/it]

كان على فاني فلونو إعادة تعيين جدول الأعمال وهي غير قادرة على إخبارنا بأية قصص اليوم.
{'not_personal': 0.407796747982502, 'personal': 0.4067214518785477, 'person': 0.36351657211780547, 'passport': 0.19601094350218773, 'Driving_license': 0.19261427223682404, 'location': 0.17228026315569878, 'health': 0.11573689430952072, 'phone': 0.1137335995833079, 'mail': 0.09674082199732463, 'birth': 0.09138355404138565, 'Tax_file_number': 0.058184199035167694, 'cookie': 0.04686965560540557, 'social_security_number': 0.032113295048475266} {} 



100%|██████████| 10/10 [03:33<00:00, 21.34s/it]

فاني فلونو هنا وستتحدث إلينا عن قصص طفولتها التي نشأت في أوغستا ، جورجيا.
{'person': 0.8056835949420929, 'personal': 0.4525861658155918, 'location': 0.41296906024217606, 'passport': 0.10641976655460894, 'Driving_license': 0.10464398446492851, 'birth': 0.09195879846811295, 'not_personal': 0.08923469266543786, 'phone': 0.033841600020726524, 'mail': 0.013618535051743189, 'cookie': 0.0037779334816150367, 'Tax_file_number': 0.0019945672247558832, 'health': 0.0005931707516234989, 'social_security_number': 0.00037172119482420385} {} 



In [6]:

pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for row in  tqdm((df_en['sentence2'][10:20]),total=10):
    txt=nltk.sent_tokenize(row)
    for sent in txt:
        # text_translated=translate(sent)
        result_details=pipeline(sent, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        print(sent)
        print(result,pii(sent),'\n')

  0%|          | 0/10 [00:00<?, ?it/s]

I won't talk about that, even though there's a lot to cover.


 10%|█         | 1/10 [00:07<01:09,  7.76s/it]

{'not_personal': 0.9388590057690939, 'personal': 0.8152696371078492, 'health': 0.22094861790537834, 'person': 0.16092061791568996, 'location': 0.1316453032195568, 'Driving_license': 0.08883541543036699, 'passport': 0.08257262594997883, 'Tax_file_number': 0.06034628674387932, 'mail': 0.036909457917014755, 'phone': 0.036280778547128044, 'birth': 0.02862405776977539, 'social_security_number': 0.02776450477540493, 'cookie': 0.021621993742883205} {} 



 20%|██        | 2/10 [00:14<00:57,  7.14s/it]

...

I don't know why that happened.
{'not_personal': 0.3643004720409711, 'personal': 0.31799039244651794, 'Driving_license': 0.24749192595481873, 'person': 0.2245122916996479, 'passport': 0.2058627912774682, 'location': 0.13380054756999016, 'mail': 0.11642533416549365, 'Tax_file_number': 0.09967575967311859, 'health': 0.0665092213700215, 'phone': 0.0645840714375178, 'cookie': 0.05060127470642328, 'social_security_number': 0.03660659119486809, 'birth': 0.024572858586907387} {} 



 90%|█████████ | 9/10 [00:56<00:06,  6.09s/it]

Fannie Flono had to reschedule and is unable to tell us any stories today.
{'person': 0.5641347199678421, 'personal': 0.4508292108774185, 'not_personal': 0.43189521382252377, 'passport': 0.2689362671226263, 'Driving_license': 0.25633750669658184, 'location': 0.231398306787014, 'phone': 0.13890684147675833, 'Tax_file_number': 0.10201560705900192, 'mail': 0.08643491566181183, 'birth': 0.06285706162452698, 'health': 0.047752173307041325, 'social_security_number': 0.033242277801036835, 'cookie': 0.030549906427040696} {} 



100%|██████████| 10/10 [01:03<00:00,  6.30s/it]

Fannie Flono is here and she is going to speak to us about her childhood stories from growing up in Augusta, GA.
{'person': 0.93558669090271, 'personal': 0.44250903762876986, 'location': 0.4311541598290205, 'birth': 0.11698594689369202, 'passport': 0.10199196822941303, 'Driving_license': 0.09949446376413107, 'not_personal': 0.09323782197316177, 'phone': 0.02916531963273883, 'mail': 0.019156261036793392, 'cookie': 0.004985143430531025, 'Tax_file_number': 0.0031286212615668774, 'health': 0.0007982385432114825, 'social_security_number': 0.0004534236213658005} {} 

